In [2]:
import numpy as np
from numpy import random as rd
import tifffile as tf
from pathlib import Path

In [3]:
rootdir = "/home/austin/data/MIT_collab/exported_data/L_batch1/"
files = [fp for fp in Path(rootdir).glob("*.tiff*")]

In [4]:
is_single_channel = False
channels = ['Treg','CD8','nuc','DC']

In [5]:
tile_size = 2048
padding = 0.0 # Add

for f in files:
    
    if not is_single_channel:
        expdir = Path(rootdir, f.name[:-5])
        try:
            Path.mkdir(expdir)
        except FileExistsError:
            pass
    else:
        expdir = rootdir
        
    data = tf.TiffFile(f).asarray()
    
    if len(data.shape) > 2:
        if is_single_channel:
            if len(np.where(data[:,:,0]-data[:,:,1])[0]) != 0 :
                raise('Split channels')
            else:
                image_data = data[:,:,0]
                w,h=data.shape
                c=0
        else:
            c,w,h = data.shape

    
    rows = list(np.arange(0,w,tile_size))
    cols = list(np.arange(0,h,tile_size))
    
    for ch in np.arange(c):
        
        fov = 0
        
        if not is_single_channel:
            image_data = data[ch,:,:]

        for x in rows:

            if w-x < tile_size:
                x_end = w
            else:
                x_end = x+tile_size

            for y in cols:

                # create empty tile, useful for padding out incomplete tiles at the edges with zeros
                tile = np.zeros((tile_size, tile_size))

                if h-y < tile_size:
                    y_end = h
                else:
                    y_end = y+tile_size

                tile[0:x_end-x, 0:y_end-y] = image_data[x:x_end, y:y_end]

                savedir = Path(expdir, 'fov' + str(fov))

                try:
                    Path.mkdir(savedir)
                except FileExistsError:
                    pass

                if is_single_channel:
                    tf.imsave(str(Path(savedir, f.name.split(".")[0] + ".tiff")), tile)
                else:
                    tf.imsave(str(Path(savedir, channels[ch] + ".tiff")), tile)

                with open(Path(expdir, "tile_metadata.txt"), "a") as metadata_f:
                    metadata_f.write(",".join(["fov" + str(fov), str(x), str(x_end), str(y), str(y_end)]) + "\n")

                fov += 1

In [4]:
czi.

'BSCMYX'